In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime 
import selenium
import re
import pyodbc
# Define the column names as a list
columns = ['MMYYYY','Bank_name', 'CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M','Onl_1Y','Onl_2Y']

Update_inst = pd.DataFrame(columns=columns)
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y


In [2]:
# import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
#url list
url_ACB = 'https://www.acb.com.vn/lai-suat-tien-gui' # Done
url_MB = 'https://www.mbbank.com.vn/Fee' #Done
url_TPB = 'https://tpb.vn/cong-cu-tinh-toan/lai-suat' #Done
url_SCB = 'https://www.scb.com.vn/vie/lai-suat'#Done 60% of the time
url_LPB = 'https://lpbank.com.vn/lai-suat-2/' #Done
url_SHB = 'https://ibanking.shb.com.vn/Rate/TideRate' #Done
url_VIB = 'https://www.vib.com.vn/vn/tiet-kiem/bieu-lai-suat-tiet-kiem-tai-quay' #Done
url_VIB_ONL = 'https://www.vib.com.vn/vn/tiet-kiem' # Done
url_list = [url_ACB, url_MB, url_TPB, url_SCB]

#Chrome location
url_chrome = 'D:/Chromedriver/chromedriver.exe'

In [4]:
SBV_int_url = 'https://www.sbv.gov.vn/webcenter/portal/vi/menu/rm/ls?_afrLoop=34719139833966466#%40%3F_afrLoop%3D34719139833966466%26centerWidth%3D80%2525%26leftWidth%3D20%2525%26rightWidth%3D0%2525%26showFooter%3Dfalse%26showHeader%3Dfalse%26_adf.ctrl-state%3Db21pu33qu_53'
response = requests.get(SBV_int_url)
allsoup = BeautifulSoup(response.text, 'html') 
rows = allsoup.find_all('tr')

# Initialize a list to store the scraped data
data = []

# Refinancing rate
for row in rows:
    cells = row.find_all('td')
    if cells:
        row_data = [cell.get_text(strip=True) for cell in cells]
        data.append(row_data)

InterBank_tb = pd.DataFrame(data[14:21])
Refinancing_tb = pd.DataFrame(data[6:8])


def convert_to_float(value):
    return float(value.replace(',', '.').replace('%', ''))

Update_inst.loc[0, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[0, 'Bank_name'] = 'SBV_InterBank'
Update_inst.loc[0, "CASA_int"] = convert_to_float(InterBank_tb.iloc[0,1])
Update_inst.loc[0, "Off_1M"] = convert_to_float(InterBank_tb.iloc[3,1])
Update_inst.loc[0, "Off_3M"] = convert_to_float(InterBank_tb.iloc[4,1])
Update_inst.loc[0, "Off_6M"] = convert_to_float(InterBank_tb.iloc[5,1])
Update_inst.loc[1, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[1, 'Bank_name'] = 'SBV_DiscountRate'
Update_inst.loc[1, 'CASA_int'] = convert_to_float(Refinancing_tb.iloc[0,1])
Update_inst.loc[2, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[2, 'Bank_name'] = 'SBV_RefinancingRate'
Update_inst.loc[2, 'CASA_int'] = convert_to_float(Refinancing_tb.iloc[1,1])

#Update_inst


In [7]:
MacroE_url = 'https://tradingeconomics.com/vietnam/inflation-cpi'

chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
MacroE = driver.get(MacroE_url)
driver.implicitly_wait(5)
Content = driver.page_source
soup = BeautifulSoup(Content, 'html.parser')

table = soup.find_all('tbody')[1]

# Tạo danh sách trống để lưu dữ liệu
data = []

# Lặp qua các hàng trong bảng
for row in table.find_all('tr'):
    # Lấy tất cả các cột trong mỗi hàng
    columns = row.find_all('td')
    # Lấy nội dung của các cột và lưu vào danh sách dữ liệu
    data.append([column.get_text() for column in columns])
Macro_E = pd.DataFrame(data)


Update_inst.loc[3, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[3, 'Bank_name'] = 'Macro_Inflation_%'
Update_inst.loc[3, "CASA_int"] = Macro_E.iloc[0,1]

Update_inst.loc[4, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[4, 'Bank_name'] = 'Macro_ConsumerPriceIndex_%'
Update_inst.loc[4, "CASA_int"] = Macro_E.iloc[1,1]

Update_inst.loc[5, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[5, 'Bank_name'] = 'Macro_ProducerPrices_Point'
Update_inst.loc[5, "CASA_int"] = Macro_E.iloc[8,1]

driver.close()

Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,06/11/2023,SBV_InterBank,0.99,1.82,3.71,5.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06/11/2023,SBV_DiscountRate,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06/11/2023,SBV_RefinancingRate,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06/11/2023,Macro_Inflation_%,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06/11/2023,Macro_ConsumerPriceIndex_%,113.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,06/11/2023,Macro_ProducerPrices_Point,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#ACB
# Function to convert a string containing ',' to a float and replace ',' with '.'
#def convert_to_float(value):
#    return float(value.replace(',', '.'))

# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
acb = driver.get(url_ACB)

# Set the common values for 'MMYYYY' and 'Bank_name'
Update_inst.loc[6, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[6, 'Bank_name'] = 'ACB'

# Check if the request was successful (status code 200)
#if acb.status_code == 200:
#driver timeout to stop forever loading
driver.implicitly_wait(30)

#get page html content
ACB_html_content = driver.page_source
    
    # Parse the HTML content using BeautifulSoup
ACB_soup = BeautifulSoup(ACB_html_content, 'html.parser')
    
    # Find the CASA_int value and convert it to float
casa = ACB_soup.find_all('span', class_='font6')[0].text
Update_inst.loc[0, 'CASA_int'] = convert_to_float(casa)

    # Find and convert offline and online values to float using list comprehension
offline_values = [convert_to_float(ACB_soup.find_all("tbody")[1].find_all("tr")[x].find_all("td", class_="xl68")[0].text.strip()) for x in [5, 7, 10, 12,16]]
online_values = [convert_to_float(ACB_soup.find_all("tbody")[3].find_all("tr")[2].find_all("td")[y].text.strip()) for y in [2, 4, 5, 6]]


    # Update the DataFrame with the converted values
Update_inst.loc[6, ["Off_1M", "Off_3M", "Off_6M", "Off_1Y", "Off_2Y"]] = offline_values
Update_inst.loc[6, ["Onl_1M", "Onl_3M", "Onl_6M", "Onl_1Y"]] = online_values
Update_inst.loc[6, ["Onl_2Y"]] = None


#close driver
driver.close()

#show result
#Update_inst

In [11]:
#MB
# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
MB = driver.get(url_MB)

#get page html content
MB_html = driver.page_source
MB_soup = BeautifulSoup(MB_html, 'html')



#wait to see if table can be retrieved
try:
    elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.CLASS_NAME, 'toggle-content')))
    print('got it')
except:
    driver.close()
    print('did not find table')


#Finding elements from table
MB_div = MB_soup.find ('div', class_= "toggle-content")
MB_table = MB_div.find_all('table')[0]


#Data processing
MB_data = []
for row in MB_table.find_all('tr'):
    columns = row.find_all('td')
    MB_data.append([column.get_text() for column in columns])


#Insert data into dataframe
MB_df = pd.DataFrame(MB_data, columns = ['Kỳ hạn', 'Lãi trả sau', 'Lãi trả trước', 'Lãi trả hàng năm'])


#Process and insert data
MB_row_raw = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'MB',
    'CASA_int': MB_df.loc[1, 'Lãi trả sau'],
    'Off_1M': MB_df.loc[5, 'Lãi trả sau'],
    'Off_3M': MB_df.loc[7, 'Lãi trả sau'],
    'Off_6M': MB_df.loc[10, 'Lãi trả sau'],
    'Off_1Y': MB_df.loc[16, 'Lãi trả sau'],
    'Off_2Y': MB_df.loc[20, 'Lãi trả sau'],
    'Onl_1M': None,
    'Onl_3M': None,
    'Onl_6M': None,
    'Onl_1Y': None,
    'Onl_2Y': None
}

for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y']:
        MB_row_raw[column] = float(MB_row_raw[column].replace('%', ''))
        MB_row_df = pd.DataFrame([MB_row_raw])

Update_inst = pd.concat([Update_inst, MB_row_df], ignore_index=True)



#driver.close()


Update_inst

did not find table


AttributeError: 'NoneType' object has no attribute 'find_all'

In [12]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,06/11/2023,SBV_InterBank,0.05,1.82,3.71,5.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06/11/2023,SBV_DiscountRate,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06/11/2023,SBV_RefinancingRate,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06/11/2023,Macro_Inflation_%,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06/11/2023,Macro_ConsumerPriceIndex_%,113.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,06/11/2023,Macro_ProducerPrices_Point,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,06/11/2023,ACB,NaN,3.3,3.5,4.8,5.3,5.4,3.5,3.7,5.0,5.1,None
7,06/11/2023,MB,0.1,3.4,3.7,5.0,5.3,6.5,None,None,None,None,None


In [14]:
#TPB
# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
TPB = driver.get(url_TPB)

#get page html content
TPB_html = driver.page_source
TPB_soup = BeautifulSoup(TPB_html, 'html')


#wait to see if table can be retrieved
try:
    elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.ID, 'tab-1')))
    print('got it')
except:
    driver.close()
    print('did not find table')


#Finding elements from table
TPB_div = TPB_soup.find ('div', class_= "tab-table tabside-bar")
TPB_table = TPB_div.find_all('table')[0]


#Data processing
TPB_data = []
for row in TPB_table.find_all('tr'):
    columns = row.find_all('td')
    TPB_data.append([column.get_text() for column in columns])


#Insert data into dataframe
TPB_df = pd.DataFrame(TPB_data, columns = ['Kỳ hạn', 'TK An Lộc', 'TK Online', 'TK Offline'])


#Process and insert data
TPB_row_raw = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'TPB',
    'CASA_int': None,
    'Off_1M': TPB_df.loc[1, 'TK Offline'],
    'Off_3M': TPB_df.loc[2, 'TK Offline'],
    'Off_6M': TPB_df.loc[3, 'TK Offline'],
    'Off_1Y': TPB_df.loc[4, 'TK An Lộc'],
    'Off_2Y': TPB_df.loc[6, 'TK An Lộc'],
    'Onl_1M': TPB_df.loc[1, 'TK Online'],
    'Onl_3M': TPB_df.loc[2, 'TK Online'],
    'Onl_6M': TPB_df.loc[3, 'TK Online'],
    'Onl_1Y': TPB_df.loc[4, 'TK Online'],
    'Onl_2Y': TPB_df.loc[6, 'TK Online']
}

for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_1Y', 'Onl_2Y']:
        TPB_row_df = pd.DataFrame([TPB_row_raw])

Update_inst = pd.concat([Update_inst, TPB_row_df], ignore_index=True)


#close driver
driver.close()

#Update_inst

did not find table


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF7047D2642+53186]
	(No symbol) [0x00007FF704742C62]
	(No symbol) [0x00007FF7045FA54D]
	(No symbol) [0x00007FF70462FE69]
	(No symbol) [0x00007FF7046314C4]
	GetHandleVerifier [0x00007FF704B25DC2+3540802]
	GetHandleVerifier [0x00007FF704B7BE6E+3893230]
	GetHandleVerifier [0x00007FF704B74223+3861411]
	GetHandleVerifier [0x00007FF70486E466+691686]
	(No symbol) [0x00007FF70474DC08]
	(No symbol) [0x00007FF704749D04]
	(No symbol) [0x00007FF704749E32]
	(No symbol) [0x00007FF70473A283]
	BaseThreadInitThunk [0x00007FF846F37614+20]
	RtlUserThreadStart [0x00007FF847C426A1+33]


In [15]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,06/11/2023,SBV_InterBank,0.05,1.82,3.71,5.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06/11/2023,SBV_DiscountRate,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06/11/2023,SBV_RefinancingRate,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06/11/2023,Macro_Inflation_%,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06/11/2023,Macro_ConsumerPriceIndex_%,113.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,06/11/2023,Macro_ProducerPrices_Point,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,06/11/2023,ACB,NaN,3.3,3.5,4.8,5.3,5.4,3.5,3.7,5.0,5.1,None
7,06/11/2023,MB,0.1,3.4,3.7,5.0,5.3,6.5,None,None,None,None,None
8,06/11/2023,TPB,None,3.80,4.00,4.80,5.55,6.30,3.80,4.00,5.00,5.55,6.30
9,06/11/2023,TPB,None,3.80,4.00,4.80,5.55,6.30,3.80,4.00,5.00,5.55,6.30


In [ ]:
#LPB
# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
LPB = driver.get(url_LPB)


#get page html content
LPB_html = driver.page_source
LPB_soup = BeautifulSoup(LPB_html, 'html')


#wait to see if table can be retrieved
try:
    elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="Content"]/div/div/div/div[1]/div/div/div[1]/div/div/div/div/div/div/div[3]/div[1]/div/table[2]')))
    print('got it')
except:
    driver.close()
    print('did not find table')


#Finding elements from table
LPB_div = LPB_soup.find ('div', class_= "tabkhcn")
LPB_table_CASA = LPB_div.find_all('table')[0]
LPB_table_OFF = LPB_div.find_all('table')[1]
LPB_table_ONL = LPB_div.find_all('table')[3]


#Data processing
LPB_data_CASA = []
for row in LPB_table_CASA.find_all('tr'):
    columns = row.find_all('td')
    LPB_data_CASA.append([column.get_text() for column in columns])
LPB_data_OFF = []
for row in LPB_table_OFF.find_all('tr'):
    columns = row.find_all('td')
    LPB_data_OFF.append([column.get_text() for column in columns])
LPB_data_ONL = []
for row in LPB_table_ONL.find_all('tr'):
    columns = row.find_all('td')
    LPB_data_ONL.append([column.get_text() for column in columns])


#Insert data into dataframe
LPB_df_CASA = pd.DataFrame(LPB_data_CASA, columns = ['STT', 'Lãi VND', 'Lãi USD'])
LPB_df_OFF = pd.DataFrame(LPB_data_OFF, columns = ['Kỳ hạn', 'Lãi trả trước', 'Lãi trả tháng', 'Lãi trả quý', 'Lãi cuối kỳ', 'Lãi USD'])
LPB_df_ONL = pd.DataFrame(LPB_data_ONL, columns = ['Kỳ hạn', 'Lãi trả trước', 'Lãi trả tháng', 'Lãi trả quý', 'Lãi cuối kỳ'])



#Process and insert data
LPB_row_raw = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'LPB',
    'CASA_int': LPB_df_CASA.loc[1, 'Lãi VND'],
    'Off_1M': LPB_df_OFF.loc[5, 'Lãi cuối kỳ'],
    'Off_3M': LPB_df_OFF.loc[7, 'Lãi cuối kỳ'],
    'Off_6M': LPB_df_OFF.loc[10, 'Lãi cuối kỳ'],
    'Off_1Y': LPB_df_OFF.loc[16, 'Lãi cuối kỳ'],
    'Off_2Y': LPB_df_OFF.loc[21, 'Lãi cuối kỳ'],
    'Onl_1M': LPB_df_ONL.loc[5, 'Lãi cuối kỳ'],
    'Onl_3M': LPB_df_ONL.loc[7, 'Lãi cuối kỳ'],
    'Onl_6M': LPB_df_ONL.loc[10, 'Lãi cuối kỳ'],
    'Onl_1Y': LPB_df_ONL.loc[16, 'Lãi cuối kỳ'],
    'Onl_2Y': LPB_df_ONL.loc[21, 'Lãi cuối kỳ']
}

for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_1Y', 'Onl_2Y']:
        LPB_row_df = pd.DataFrame([LPB_row_raw])

Update_inst = pd.concat([Update_inst, LPB_row_df], ignore_index=True)


#close driver
driver.close()

#Update_inst

got it


In [ ]:
#SHB
# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
SHB = driver.get(url_SHB)


#get page html content
SHB_html = driver.page_source
SHB_soup = BeautifulSoup(SHB_html, 'html')


#wait to see if table can be retrieved
try:
    elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[6]/div[3]/div[1]/div/table')))
    print('got it')
except:
    driver.close()
    print('did not find table')



#Finding elements from table
SHB_table = SHB_soup.find('table', class_= "table table-bordered")



#Data processing
SHB_data = []
for row in SHB_table.find_all('tr'):
    columns = row.find_all('td')
    SHB_data.append([column.get_text() for column in columns])



#Insert data into dataframe
SHB_df = pd.DataFrame(SHB_data, columns = ['Kỳ hạn', 'VND', 'USD', 'EUR'])



#Process and insert data
SHB_row_raw = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'SHB',
    'CASA_int': None,
    'Off_1M': None,
    'Off_3M': None,
    'Off_6M': None,
    'Off_1Y': None,
    'Off_2Y': None,
    'Onl_1M': SHB_df.loc[1, 'VND'],
    'Onl_3M': SHB_df.loc[3, 'VND'],
    'Onl_6M': SHB_df.loc[6, 'VND'],
    'Onl_1Y': SHB_df.loc[12, 'VND'],
    'Onl_2Y': SHB_df.loc[15, 'VND']
}

for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_1Y', 'Onl_2Y']:
        SHB_row_df = pd.DataFrame([SHB_row_raw])

Update_inst = pd.concat([Update_inst, SHB_row_df], ignore_index=True)


#close driver
driver.close()

#Update_inst

got it


In [ ]:
#VIB
# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
VIB = driver.get(url_VIB)


#get page html content
VIB_html_OFF = driver.page_source
VIB_soup_OFF = BeautifulSoup(VIB_html_OFF, 'html')


#wait to see if table can be retrieved
try:
    elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="saving-table-TDE"]')))
    print('got offline table')
except:
    driver.close()
    print('did not find table')



#Finding elements from table
VIB_div_OFF = VIB_soup_OFF.find('div', class_='vib-v2-right-box-table-expression small-height')
VIB_table_OFF = VIB_div_OFF.find('div', class_= 'bx-viewport')


#Data processing
VIB_data_OFF = []
for row in VIB_table_OFF.find_all('div', class_='number_rate_hover'):
    columns = row.find_all('span')
    VIB_data_OFF.append([column.get_text() for column in columns])



#Insert data into dataframe
VIB_df_OFF = pd.DataFrame(VIB_data_OFF, columns = ['<300tr_OFF'])
driver.close()



#VIB_ONL
# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
VIB_ONL = driver.get(url_VIB_ONL)



#get page html content
VIB_html_ONL = driver.page_source
VIB_soup_ONL = BeautifulSoup(VIB_html_ONL, 'html')


#wait to see if table can be retrieved
try:
    elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="saving-table-term"]/div[2]/div/div[1]/div/div[1]/div[1]/div')))
    print('got online table')
except:
    driver.close()
    print('did not find table')


#Finding elements from table
VIB_div_ONL = VIB_soup_ONL.find('div', class_='vib-v2-right-box-table-expression small-height')
VIB_table_ONL = VIB_div_ONL.find('div', class_= 'bx-viewport')


#Data processing
VIB_data_ONL = []
for row in VIB_table_ONL.find_all('div', class_='number_rate_hover'):
    columns = row.find_all('span')
    VIB_data_ONL.append([column.get_text() for column in columns])



#Insert data into dataframe
VIB_df_ONL = pd.DataFrame(VIB_data_ONL, columns = ['<300tr_ONL'])


#Process and insert data
VIB_row_raw = {
    'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
    'Bank_name': 'VIB',
    'CASA_int': None, #VIB_df_OFF.loc[3, '<300tr_OFF'],
    'Off_1M': VIB_df_OFF.loc[1, '<300tr_OFF'],
    'Off_3M': VIB_df_OFF.loc[5, '<300tr_OFF'],
    'Off_6M': VIB_df_OFF.loc[2, '<300tr_OFF'],
    'Off_1Y': VIB_df_OFF.loc[11, '<300tr_OFF'],
    'Off_2Y': VIB_df_OFF.loc[14, '<300tr_OFF'],
    'Onl_1M': VIB_df_ONL.loc[1, '<300tr_ONL'],
    'Onl_3M': VIB_df_ONL.loc[5, '<300tr_ONL'],
    'Onl_6M': VIB_df_ONL.loc[2, '<300tr_ONL'],
    'Onl_1Y': None,
    'Onl_2Y': VIB_df_ONL.loc[13, '<300tr_ONL']
}

for column in ['Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_2Y']:
        VIB_row_raw[column] = float(VIB_row_raw[column].replace('%', ' '))
        VIB_row_df = pd.DataFrame([VIB_row_raw])

Update_inst = pd.concat([Update_inst, VIB_row_df], ignore_index=True)


#close driver
driver.close()

#Update_inst

got offline table
got online table


In [8]:
#SCB
# Send an HTTP GET request to the URL and retrieve the content
off_chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=off_chrome_options)
driver.maximize_window()
SCB = driver.get(url_SCB)


# #button click
# try:
#     on_button = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="btn_view"]')))
#     TKOn = driver.find_element(by = By.XPATH, value='//*[@id="sl_number"]/option[4]')
#     TKOn.click()
#     Online_Button = driver.find_element(by = By.ID, value='btn_view')
#     Online_Button.click()
#     print ('on_button clicked')
# except:
#     driver.quit()
#     print ('on_button not clicked')



# #Wait for table top load
# try:
#     on_elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="Form1News"]/section[2]/div/div/div[2]/div[2]/table/thead/tr[1]/th[1]')))
#     print ('got online')
# except:
#     driver.quit()
#     print ('did not find online')


# #get page html content
# SCB_html_on = driver.page_source
# SCB_soup_on = BeautifulSoup(SCB_html_on, 'html') #problem installing html5lib
# #Finding elements from table
# SCB_div_on = SCB_soup_on.find('div', class_= "access-details")
# SCB_table_on = SCB_div_on.find_all('table')[0]
# #Data processing
# SCB_data_on = []
# for on_row in SCB_table_on.find_all('tr'):
#     on_columns = on_row.find_all('td')
#     SCB_data_on.append([on_column.get_text() for on_column in on_columns])
# #Insert data into dataframe
# SCB_df_on = pd.DataFrame(SCB_data_on, columns = ['Kỳ hạn', 'Lãi trước kỳ', 'Lãi hàng tháng', 'Lãi cuối kỳ'])
# driver.quit()



#Offline Table
on_chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=on_chrome_options)
driver.maximize_window()
SCB_off = driver.get(url_SCB)
#Button click
try:
    on_button = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="btn_view"]')))
    TKOff = driver.find_element(by = By.XPATH, value='//*[@id="sl_number"]/option[6]')
    TKOff.click()
    Offline_Button = driver.find_element(by = By.ID, value='btn_view')
    Offline_Button.click()
    print ('off_button clicked')
except:
    driver.quit()
    print ('off_button not clicked')

#Wait for table top load
try:
    off_elem = WebDriverWait(driver, 60).until (EC.presence_of_element_located((By.XPATH, '//*[@id="Form1News"]/section[2]/div/div/div[2]/div[2]/table[1]/thead/tr/th[1]')))
    print ('got offline')
except:
    driver.quit()
    print ('did not find offline')

#get page html content
SCB_html_off = driver.page_source
SCB_soup_off = BeautifulSoup(SCB_html_off, 'html')
#Finding elements from table
SCB_div_off = SCB_soup_off.find('div', class_= "access-details")
SCB_table_off = SCB_div_off.find_all('table')[0]
#Data processing
SCB_data_off = []
for row in SCB_table_off.find_all('tr'):
    columns = row.find_all('td')
    SCB_data_off.append([column.get_text() for column in columns])
#Insert data into dataframe
SCB_df_off = pd.DataFrame(SCB_data_off, columns = ['Kỳ hạn', 'Lãi cuối kỳ', 'Lãi hàng năm', 'Lãi 6 tháng', 'Lãi hàng quý', 'Lãi hàng tháng', 'Lãi trước'])

# #Process and insert data
# SCB_row_raw = {
#     'MMYYYY': datetime.datetime.now().strftime("%d/%m/%Y"),
#     'Bank_name': 'SCB',
#     'CASA_int': SCB_df_off.loc[1, 'Lãi hàng tháng'],
#     'Off_1M': SCB_df_off.loc[3, 'Lãi cuối kỳ'],
#     'Off_3M': SCB_df_off.loc[5, 'Lãi cuối kỳ'],
#     'Off_6M': SCB_df_off.loc[8, 'Lãi cuối kỳ'],
#     'Off_1Y': SCB_df_off.loc[14, 'Lãi cuối kỳ'],
#     'Off_2Y': SCB_df_off.loc[17, 'Lãi cuối kỳ'],
#     'Onl_1M': SCB_df_on.loc[4, 'Lãi cuối kỳ'],
#     'Onl_3M': SCB_df_on.loc[4, 'Lãi cuối kỳ'],
#     'Onl_6M': TPB_df.loc[6, 'TK Online'],
#     'Onl_1Y': SCB_df_on.loc[15, 'Lãi cuối kỳ'],
#     'Onl_2Y': SCB_df_on.loc[19, 'Lãi cuối kỳ']
# }

# for column in ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_1Y', 'Onl_2Y']:
#         SCB_row_raw[column] = float(SCB_row_raw[column].replace(',', '.'))
#         SCB_row_df = pd.DataFrame([SCB_row_raw])

# Update_inst = pd.concat([Update_inst, SCB_row_df], ignore_index=True)

# #close driver
# driver.quit()

#Update_inst

off_button not clicked
did not find offline


MaxRetryError: HTTPConnectionPool(host='localhost', port=50682): Max retries exceeded with url: /session/9dfb91553f5fdec3e466d92a81c7b14e/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002239B749810>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:

Bank_int = pd.melt (Update_inst.replace(np.nan, None), id_vars = ['MMYYYY', 'Bank_name'], value_vars= ['CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M', 'Onl_1Y', 'Onl_2Y'])
Bank_int

,MMYYYY,Bank_name,variable,value
0,01/11/2023,SBV_InterBank,CASA_int,0.05
1,01/11/2023,SBV_DiscountRate,CASA_int,3.0
2,01/11/2023,SBV_RefinancingRate,CASA_int,4.5
3,01/11/2023,Macro_Inflation_%,CASA_int,3.59
4,01/11/2023,Macro_ConsumerPriceIndex_%,CASA_int,113.38
...,...,...,...,...
138,01/11/2023,TPB,Onl_2Y,6.30
139,01/11/2023,LPB,Onl_2Y,6.40
140,01/11/2023,SHB,Onl_2Y,6.6
141,01/11/2023,VIB,Onl_2Y,5.9


In [ ]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,01/11/2023,SBV_InterBank,0.05,1.65,4.51,6.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/11/2023,SBV_DiscountRate,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/11/2023,SBV_RefinancingRate,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/11/2023,Macro_Inflation_%,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/11/2023,Macro_ConsumerPriceIndex_%,113.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01/11/2023,Macro_ProducerPrices_Point,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,01/11/2023,ACB,NaN,3.3,3.5,4.8,5.3,5.4,3.5,3.7,5.0,5.1,None
7,01/11/2023,MB,0.1,3.4,3.7,5.0,5.3,6.5,None,None,None,None,None
8,01/11/2023,TPB,None,3.80,4.00,4.80,5.55,6.30,3.80,4.00,5.00,5.55,6.30
9,01/11/2023,LPB,0.20,3.50,3.70,4.80,5.30,6.10,3.80,4.00,5.10,5.60,6.40


In [ ]:
server = '10.16.157.42' 
database = 'USER_DATA'  
cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:

for index, row in Bank_int.iterrows():
     cursor.execute("INSERT INTO USER_DATA.LANDT19.BANK_INTEREST (BUSINESS_DATE,BANK_NAME,INT_TYPE,INT_VALUE) values(?,?,?,?)", row.MMYYYY, row.Bank_name, row.variable, row.value)
cnxn.commit()
cursor.close()
